# Examen Enero 2023

### Bioinformática y Análisis Genómico
### Grado en Bioquímica
### Universidad de Sevilla

[Keras](https://keras.io/) es una librería de Python que permite crear de forma sencilla redes neuronales y se encuentra dentro de la plataforma [TensorFlow](https://www.tensorflow.org/). Para instalarlo, basta utilizar la siguiente línea de código (descomentándola):

In [1]:
# pip install tensorflow

Además, en este examen se utilizarán los paquetes `numpy`, `pandas` y `sklearn`, que se deberás tener instalados.

In [2]:
# pip install numpy
# pip install pandas
# pip install -U scikit-learn

Además, como se van a emplear números pseudoaletorios durante el ejercicio, hay que definir una semilla para que el análisis sea reproducible.

In [3]:
from tensorflow import random as tensorflow_random

tensorflow_random.set_seed(394867)

Después hay que importar los paquetes que van a permitir la división del conjunto de datos en entrenamiento y test y definir la semilla para numpy (paquete para trabajar con arrays).

In [4]:
import numpy
import pandas
from sklearn import model_selection

numpy.random.seed(43958734)
numpy.set_printoptions(threshold=10)

Finalmente importamos el paquete `keras`.

In [5]:
from tensorflow import keras

## Ejercicio de redes naturales

Se proporciona un fichero de datos ómicos con el nombre `keras_cancer.tsv`. Este fichero recoge información de expresión génica y miRNAs de 220 pacientes de cáncer de mama y la clasificación de cada uno según el tipo de cáncer que presenta. De esta forma, las primeras 200 columnas contienen datos de mRNAs (nombradas como los genes a los que corresponde el transcrito), las 184 siguientes de miRNAs (nombradas como los miRNAs cuantificados, con el identificador hsa-número) y la última columna, type, contiene información del tipo de cáncer, donde un valor de 1 se asocia a Basal, uno de 2 a Her2 y uno de 3 a LumA. Usa read_csv de pandas para leer los datos, almacénalos en una variable llamada breast y pre-visualízalos con head. 

In [6]:
breast = pandas.read_csv("keras_cancer.tsv", sep="\t")
breast.head()
# Comprobamos que los datos se han leído correctamente

,RTN2,NDRG2,CCDC113,FAM63A,ACADS,GMDS,HLA-H,SEMA4A,ETS2,LIMD2,...,hsa-mir-9-2,hsa-mir-92a-1,hsa-mir-92a-2,hsa-mir-92b,hsa-mir-93,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b,type
0,4.362183,7.533461,3.956124,4.457170,2.256817,6.017940,5.006907,3.217812,4.734446,5.099598,...,12.779765,11.320936,15.288781,5.832308,13.272791,6.920198,6.651011,8.745468,15.783732,1
1,1.984492,7.455194,5.427623,5.440957,4.028813,4.341692,6.178668,2.864659,5.411029,4.211397,...,13.823930,12.098454,14.262681,6.900878,14.086331,6.292604,6.329947,9.235832,14.902776,1
2,1.727323,8.079968,2.227300,5.543480,2.629855,6.363030,6.039563,5.946028,5.651670,3.304513,...,6.544669,9.040364,13.924606,4.563778,13.511479,5.330801,5.910797,9.333587,13.661209,1
3,4.363996,5.793750,3.544866,4.737114,4.269101,4.001104,7.087633,5.007565,5.902449,5.479451,...,11.540357,10.337624,14.004158,6.174951,12.868614,4.931573,5.409937,9.813171,14.805293,1
4,2.447562,7.158993,4.691256,4.808728,2.442135,7.029723,5.936138,5.901459,6.641225,5.508654,...,10.795053,12.018595,15.720121,8.151821,13.942631,4.109904,6.511839,8.355920,15.116468,1


Confirma que las dimensiones son correctas usando shape.

In [7]:
breast.shape
# Las dimensiones son correctas, presentando 220 filas (una por cada paciente) y 385 columnas (200 de mRNAs + 184 de miRNAs + 1 para el tipo de cáncer)

(220, 385)

Separamos en atributos de predicción y objetivo, en este caso vamos el objetivo es predecir el tipo de cáncer a partir de los datos de expresión génica. Para esto define atributos como las primeras 200 columnas de la tabla de datos usando iloc, estas serán las variables predictoras. También define objetivo como la columna identificada con type en los datos guardados en breast. Esta será la variable respuesta que se quiere predecir. Es necesario transformar estos datos a un array de Numpy, compatible con Keras, usando to_numpy(). Muestras estas variables con print.

In [19]:
atributos = breast.iloc[:,0:200].to_numpy()
objetivo = breast.iloc[:,384].to_numpy()
print(atributos)
print(objetivo)
# Se ha comprobado que se han extraído todas las columnas de mRNA correctamente en el caso de atributos (desde RTN2 hasta PBX1)

[[4.36218331 7.53346145 3.95612417 ... 3.99198186 3.35263746 7.88092301]
 [1.98449231 7.45519376 5.42762306 ... 3.34369865 3.63714211 7.80708187]
 [1.72732287 8.07996824 2.22730024 ... 3.28931416 3.63387947 7.44171248]
 ...
 [4.44270845 7.03873191 3.86784085 ... 5.4663389  4.40897879 7.79461371]
 [4.58795195 4.80163296 4.43026828 ... 5.58968014 4.53612901 8.11940302]
 [3.48796381 5.43834437 4.20031809 ... 5.26583021 2.95082991 7.41317155]]
[1 1 1 ... 3 3 3]


A continuación, es necesario definir los subconjuntos de entrenamiento y prueba (tanto para  los atributos como objetivo) para la construcción y evaluación de redes neuronales mediante aprendizaje supervisado. Aquí definiremos como conjunto de entrenamiento a un 80% de las instancias o pacientes mientras que el 20% restante se usará como conjunto de test sobre el que posteriormente se probará el modelo para evaluar su eficacia. Realiza este paso usando model_selection.train_test_split con los argumentos oportunos. Usa los siguientes nombres de variables atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento y objetivo_prueba. Muestra con print las dimensiones (shape) de cada variable definida en este paso. 

In [9]:
atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(atributos, objetivo, train_size=0.8)

In [10]:
print(atributos_entrenamiento.shape)
print(atributos_prueba.shape)
print(objetivo_entrenamiento.shape)
print(objetivo_prueba.shape)
# print(objetivo_entrenamiento)
# print(objetivo_prueba)

(176, 200)
(44, 200)
(176,)
(44,)


Para evitar problemas de desbordamiento numérico y realizar un preprocesamiento de los datos para evitar valores atípicos, se puede emplear una capa de normalización, usando la instancia de la clase keras `layers.experimental.preprocessing.Normalization`. En este caso, se pide que se guarde en una variable llamada normalizador y que se use sobre ella el método `adapt` para aplicarlo sobre los atributos de entrenamiento guardados anteriormente en la variable atributos_entrenamiento.

In [11]:
normalizador = keras.layers.experimental.preprocessing.Normalization()
normalizador.adapt(atributos_entrenamiento)

A partir de estos datos, ya se puede emplear Keras para construir la red y entrenar el modelo.  Para esto, se va a emplear un enfoque secuencial en el que se va a construir la red definiendo una por una las distintas capas. De esta forma, se comienza definiendo la primera capa (tipo de capa, número de neuronas, etc) y se van añadiendo una a una las distintas capas hasta llegar a la de salida. Almacena esta red en la variable net_breast e inicializala usando Sequential de keras. Añade las capas sucesivamente con add.

La primera capa o capa de entrada se construye con la clase `Input`, y hay que definir la forma de los datos de entrada como un array con el número de dimensiones necesario. En este caso, una dimensión de 200 atributos.

Justo después de la capa de entrada se debe añadir la capa de normalización previamente configurada que, al haber ya sido adaptada a los datos de entrada, no necesita una especificación del número de neuronas que debe contener. Esta capa se almecenó en normalizador.

Como vamos a trabajar con una red neuronal con neuronas completamente conectadas, el resto de las capas se contruyen con la clase `layers.Dense`, donde de nuevo hay que especificar la cantidad de neuronas y, en este caso, la función de activación, es decir, la función que se aplica en la salida de cada neurona tras la integración de las señales que llegan a ella. Para este problema, conviene restringirse al uso de las funciones de activación `softmax` y `sigmoid`. De esta forma añade una capa oculta con 15 neuronas y función de activación softmax. Por último, añade la capa de salida con 3 neuronas y de nuevo softmax como función de activación. 

Hay que tener en cuenta que la capa de salida debe contener un número de neuronas igual al número de clases de salida que queremos predecir (para este caso concreto, dependiendo de las funciones que se usen para definir y compilar la red esto puede variar) y que se le puede aplicar también una función de activación o dejar la función identidad que se usa por defecto.

In [12]:
# net_breast, Sequential de keras, add para añadir las capas
net_breast = keras.Sequential()
net_breast.add(keras.Input(shape=(200,)))
net_breast.add(normalizador)
net_breast.add(keras.layers.Dense(15, activation="softmax"))
net_breast.add(keras.layers.Dense(3, activation="softmax"))

El método `summary` se usa para obtener un resumen de la estructura de la red. Para cada una de las capas, indica la forma de su salida y su número de parámetros La forma de la salida se representa como un array donde el primer número muestra el número de lotes (`None` implica que se especificará más adelante) y los siguientes, el número de salidas en cada dimensión del array. En el caso de la capa de entrada, no aparece ya que no posee parámetros. En la parte inferior se muestra el número de parámetros totales y cuántos de ellos son o no entrenables. Los parámetros entrenables son los que la red aprende y modifica mediante el algoritmo de entrenamiento, que serían el peso de la conexión de cada neurona de la capa anterior con las neuronas de la capa actual más el sesgo de cada una de las neuronas de la capa.

In [13]:
net_breast.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 200)              401       
 n)                                                              
                                                                 
 dense (Dense)               (None, 15)                3015      
                                                                 
 dense_1 (Dense)             (None, 3)                 48        
                                                                 
Total params: 3,464
Trainable params: 3,063
Non-trainable params: 401
_________________________________________________________________


Para entrenar una red neuronal hay que compilarla usando compile. Para esto, se define el algoritmo de aprendizaje u optimizador (con el atributo *optimizer* seguido del nombre) y la función de pérdida (*loss*) a minimizar y, de forma opcional, las métricas que queremos que se muestren en el entrenamiento además del valor de la función de pérdida (*metrics*). En este caso, se recomienda usar como optimizador el descenso estocástico por el gradiente (clase `optimizers.SGD`) y modular su parámetro *learning_rate* hasta obtener el rendimiento deseado (por defecto toma un valor de 0.01, se sugiere usar el valor 0.07). Este parámetro define la velocidad a la que una red puede modificar sus parámetros entrenables a lo largo de los distintos pasos de entrenamiento, de manera que mayores valores producen un cambio mayor. Se recomienda usar como función de pérdida (*loss*) *categorical_crossentropy* y como métrica (*metrics*) *accuracy*.

In [14]:
net_breast.compile(optimizer=keras.optimizers.SGD(learning_rate=0.07), loss="sparse_categorical_crossentropy", metrics="accuracy")

Una vez compilada la red con la información anterior, sólo falta entrenarla usando el método `fit` sobre nuestro conjunto de entrenamiento, definiendo los atributos por un lado y el objetivo o salida por otro.

El optimizador es el que se conoce como algoritmo de retropropagación de la red, se usa para actualizar los pesos y los sesgos. En lugar de realizar esta actualización con todos los ejemplos de entrenamiento, se pueden crear subconjuntos aleatorios de ellos y se irán actualizando tras procesar cada uno de esos subconjuntos o lotes (*batch*), de manera que cuando se hayan considerado todos los lotes, habrá transcurrido un paso completo del algoritmo, una época (*epoch*). A la hora de entrenar la red, estos parámetros tienen mucho peso, ya que definen el número de pasos de entrenamiento que estamos permitiendo para nuestra red. El tamaño de cada lote se especifica en el argumento *batch_size* y el número de épocas, en *epochs*. Recuerda que los atributos y objetivos se encuentran almacenados en atributos_entrenamiento y objetivo_entrenamiento. Se sugiera usar batch_size=5 y epochs=20.

In [15]:
net_breast.fit(atributos_entrenamiento,(objetivo_entrenamiento-1),batch_size=5,epochs=20)

Epoch 1/20
36/36 [==============================] - 1s 2ms/step - loss: 0.9752 - accuracy: 0.6591
Epoch 2/20
36/36 [==============================] - 0s 2ms/step - loss: 0.7491 - accuracy: 0.7557
Epoch 3/20
36/36 [==============================] - 0s 2ms/step - loss: 0.5899 - accuracy: 0.8068
Epoch 4/20
36/36 [==============================] - 0s 2ms/step - loss: 0.4780 - accuracy: 0.8864
Epoch 5/20
36/36 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.9489
Epoch 6/20
36/36 [==============================] - 0s 2ms/step - loss: 0.3214 - accuracy: 0.9773
Epoch 7/20
36/36 [==============================] - 0s 3ms/step - loss: 0.2685 - accuracy: 0.9830
Epoch 8/20
36/36 [==============================] - 0s 2ms/step - loss: 0.2264 - accuracy: 0.9943
Epoch 9/20
36/36 [==============================] - 0s 2ms/step - loss: 0.1943 - accuracy: 0.9943
Epoch 10/20
36/36 [==============================] - 0s 2ms/step - loss: 0.1697 - accuracy: 0.9943
Epoch 11/20
36/36 [

Una vez entrenada la red, podemos observar los valores de las métricas de precisión sobre el propio conjunto de entrenamiento (prediciendo los mismos ejemplos con los que se está entrenando). Sin embargo, puede ser que haya memorizado esos datos pero no pueda generalizar para otros nuevos, es lo que se denomina sobreajuste y se puede determinar utilizando el método `evaluate` para aplicar la red entrenada sobre el conjunto de prueba, de forma que valores de las métricas similares a las obtenidas sobre el entrenamiento indican que no existe este sobreajuste. Recuerda que el conjunto de evaluación se almacenó en atributos_prueba y objetivo_prueba.

In [16]:
net_breast.evaluate(atributos_prueba,(objetivo_prueba-1))

2/2 [==============================] - 0s 6ms/step - loss: 0.0701 - accuracy: 1.0000


[0.07014363259077072, 1.0]

Para predecir los valores de nuevas instancias, simplemente le pasamos a la red los valores de sus atributos y nos devolverá tres valores para cada uno, que serían los valores de probabilidad devueltos por cada una de las neuronas de salida, por lo que el mayor de los tres será la clase predicha por la red.

Con `argmax` obtenemos la clase de mayor probabilidad para cada instancia usando el argumento axis=1 para indicarle que mire en cada subarray. Así obtenemos que para las primeras 5 instancias del conjunto, la clase predicha es la primera (recordar que en Python los índices empiezan por el 0), que se correspondería con Basal.

In [17]:
numpy.argmax(net_breast(atributos), axis=1)

array([0, 0, 0, ..., 2, 2, 2], dtype=int64)